### Load tensorflow

In [0]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd

In [4]:
!ls

gdrive	sample_data


In [0]:
!cd gdrive

In [11]:
!ls

gdrive	mnist_dnn_v1.h5  sample_data


In [0]:
data = pd.read_csv('gdrive/My Drive/Colab Notebooks/prices.csv')

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [7]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [8]:
data.shape

(851264, 7)

In [9]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
open,851264.0,7.083699e+01,8.369588e+01,0.85,3.384000e+01,5.277000e+01,7.988000e+01,1.584440e+03
close,851264.0,7.085711e+01,8.368969e+01,0.86,3.385000e+01,5.280000e+01,7.989000e+01,1.578130e+03
low,851264.0,7.011841e+01,8.287729e+01,0.83,3.348000e+01,5.223000e+01,7.911000e+01,1.549940e+03
high,851264.0,7.154348e+01,8.446550e+01,0.88,3.419000e+01,5.331000e+01,8.061000e+01,1.600930e+03
volume,851264.0,5.415113e+06,1.249468e+07,0.00,1.221500e+06,2.476250e+06,5.222500e+06,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [11]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
X=data.iloc[0:1000]

In [13]:
X.shape

(1000, 5)

In [14]:
#Target variable 'close'
y=pd.DataFrame(X['close'])

#Drop the target variable 'close' from Features
X.drop(['close'],axis=1,inplace=True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
y.shape

(1000, 1)

In [16]:
X.head()

,open,low,high,volume
0,123.430000,122.309998,126.250000,2163600.0
1,125.239998,119.940002,125.540001,2386400.0
2,116.379997,114.930000,119.739998,2489500.0
3,115.480003,113.500000,117.440002,2006300.0
4,117.010002,114.089996,117.330002,1408600.0


In [17]:
y.head()

,close
0,125.839996
1,119.980003
2,114.949997
3,116.620003
4,114.970001


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
X_p=tf.placeholder(tf.float32,[None,4],name='Input')
x_n=tf.nn.l2_normalize(X_p,1)
y_p=tf.placeholder(tf.float32,[None,1],name='Target')

2.Define Weights and Bias

In [20]:
#We are initializing weights and Bias with Zero
w = tf.Variable(tf.zeros([4,1], dtype = tf.float32), name="Weight") 
b = tf.Variable(tf.zeros(1, dtype = tf.float32), name="bias")

Instructions for updating:
Colocations handled automatically by placer.


3.Prediction

In [0]:
y_pred=tf.add(tf.matmul(x_n,w),b)

4.Loss (Cost) Function [Mean square error]

In [0]:
loss=tf.reduce_mean(tf.square(y_pred-y_p),name='loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs=100

In [26]:
for i in range(100):
  t,train_loss=sess.run([train_op,loss],feed_dict={X_p:X_train,y_p:y_train})
  print('Current Loss on iteration', i, train_loss)


('Current Loss on iteration', 0, 3215.2407)
('Current Loss on iteration', 1, 3215.144)
('Current Loss on iteration', 2, 3215.0554)
('Current Loss on iteration', 3, 3214.9731)
('Current Loss on iteration', 4, 3214.8977)
('Current Loss on iteration', 5, 3214.8281)
('Current Loss on iteration', 6, 3214.7637)
('Current Loss on iteration', 7, 3214.705)
('Current Loss on iteration', 8, 3214.6504)
('Current Loss on iteration', 9, 3214.6)
('Current Loss on iteration', 10, 3214.5535)
('Current Loss on iteration', 11, 3214.5103)
('Current Loss on iteration', 12, 3214.4712)
('Current Loss on iteration', 13, 3214.4353)
('Current Loss on iteration', 14, 3214.4019)
('Current Loss on iteration', 15, 3214.3713)
('Current Loss on iteration', 16, 3214.3428)
('Current Loss on iteration', 17, 3214.317)
('Current Loss on iteration', 18, 3214.2922)
('Current Loss on iteration', 19, 3214.2703)
('Current Loss on iteration', 20, 3214.2493)
('Current Loss on iteration', 21, 3214.2307)
('Current Loss on iteratio

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [86]:
w=sess.run(w)
w.shape

(4, 1)

In [87]:
b=sess.run(b)
b.shape

(1,)

### Find the Absolute mean square loss difference between training and testing loss.

In [28]:
t,train_loss=sess.run([train_op,loss],feed_dict={X_p:X_train,y_p:y_train})
print('Trainig Loss : ', train_loss)


('Trainig Loss : ', 3214.0093)


In [30]:
t,test_loss=sess.run([train_op,loss],feed_dict={X_p:X_test,y_p:y_test})
print('Test Loss : ', test_loss)


('Test Loss : ', 4159.761)


In [31]:
print('Absolute mean square loss : ',abs(train_loss-test_loss))

('Absolute mean square loss : ', 945.75195)


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [32]:


#Initialize Sequential model
model = tf.keras.models.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

Instructions for updating:
Use tf.cast instead.


### Execute the model

In [33]:
model.fit(X_train, y_train, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
800/800 [==============================] - 0s 385us/sample - loss: 5886.1138
Epoch 2/100
800/800 [==============================] - 0s 121us/sample - loss: 4066.7573
Epoch 3/100
800/800 [==============================] - 0s 125us/sample - loss: 3387.4672
Epoch 4/100
800/800 [==============================] - 0s 124us/sample - loss: 3142.4388
Epoch 5/100
800/800 [==============================] - 0s 123us/sample - loss: 3016.1352
Epoch 6/100
800/800 [==============================] - 0s 124us/sample - loss: 3011.6092
Epoch 7/100
800/800 [==============================] - 0s 122us/sample - loss: 2989.2554
Epoch 8/100
800/800 [==============================] - 0s 124us/sample - loss: 2975.3136
Epoch 9/100
800/800 [==============================] - 0s 128us/sample - loss: 2988.7321
Epoch 10/100
800/800 [==============================] - 0s 125us/sample - loss: 2980.6268
Epoch 11/100
800/800 [==============================] - 0s 95

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [49]:
import numpy as np
import pandas as pd

iris=pd.read_csv('gdrive/My Drive/Colab Notebooks/11_Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
Target=pd.DataFrame(iris['Species'])
Feature=iris.drop(['Species','Id'],axis=1)

In [36]:
Target.shape

(150, 1)

In [37]:
Feature.shape

(150, 4)

In [38]:
Target.head()

,Species
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Target['Species'] = le.fit_transform(Target['Species'])

In [40]:
Target['Species'].value_counts()

2    50
1    50
0    50
Name: Species, dtype: int64

In [0]:
Target=pd.get_dummies(Target)

In [53]:
Target.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(Feature,Target,test_size=0.3)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model1 = tf.keras.models.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model1.add(tf.keras.layers.Dense(10, input_shape=(4,)))

#Add 1st hidden layer
model1.add(tf.keras.layers.Dense(8))

#Add OUTPUT layer
model1.add(tf.keras.layers.Dense(3, activation='softmax'))


In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model1.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [57]:
model1.fit(X_train,y_train,epochs=30)

Epoch 1/30
105/105 [==============================] - 0s 1ms/sample - loss: 1.9235 - acc: 0.2381
Epoch 2/30
105/105 [==============================] - 0s 174us/sample - loss: 1.3425 - acc: 0.3524
Epoch 3/30
105/105 [==============================] - 0s 202us/sample - loss: 1.2077 - acc: 0.5429
Epoch 4/30
105/105 [==============================] - 0s 215us/sample - loss: 0.6315 - acc: 0.8762
Epoch 5/30
105/105 [==============================] - 0s 196us/sample - loss: 0.5383 - acc: 0.8190
Epoch 6/30
105/105 [==============================] - 0s 170us/sample - loss: 0.6089 - acc: 0.6762
Epoch 7/30
105/105 [==============================] - 0s 170us/sample - loss: 0.4499 - acc: 0.8286
Epoch 8/30
105/105 [==============================] - 0s 175us/sample - loss: 0.8627 - acc: 0.6952
Epoch 9/30
105/105 [==============================] - 0s 155us/sample - loss: 0.3873 - acc: 0.9143
Epoch 10/30
105/105 [==============================] - 0s 175us/sample - loss: 0.6143 - acc: 0.6857
Epoch 11/30

In [58]:
y_pred=model1.predict(X_test)
y_pred

array([[1.60707750e-05, 5.35268933e-02, 9.46457088e-01],
       [9.45850089e-03, 7.46905148e-01, 2.43636340e-01],
       [1.26573385e-03, 4.56191391e-01, 5.42542875e-01],
       [2.36777421e-02, 9.49813366e-01, 2.65089776e-02],
       [8.68542993e-04, 7.11693645e-01, 2.87437856e-01],
       [9.88479316e-01, 1.15201250e-02, 6.44754664e-07],
       [6.58613804e-04, 3.97767544e-01, 6.01573825e-01],
       [1.78227145e-02, 9.56316531e-01, 2.58606970e-02],
       [3.88658489e-04, 4.59857523e-01, 5.39753854e-01],
       [2.22825420e-05, 2.08500490e-01, 7.91477203e-01],
       [9.78284180e-01, 2.17114221e-02, 4.43276895e-06],
       [9.85938609e-01, 1.40608223e-02, 6.17948388e-07],
       [6.88733999e-04, 5.06189227e-01, 4.93122071e-01],
       [2.67919240e-04, 2.96384811e-01, 7.03347325e-01],
       [9.27165210e-01, 7.28179589e-02, 1.68224069e-05],
       [2.16229193e-04, 1.21773690e-01, 8.78010035e-01],
       [9.75821078e-01, 2.41752211e-02, 3.68738006e-06],
       [6.65647312e-05, 1.21195

### Report Accuracy of the predicted values

In [0]:
y_p=np.argmax(y_pred,axis=1)

In [60]:
y_p

array([2, 1, 2, 1, 1, 0, 2, 1, 2, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 1, 2, 1,
       2, 1, 1, 2, 2, 0, 2, 2, 1, 2, 1, 1, 0, 0, 2, 1, 0, 2, 0, 0, 0, 1,
       2])

In [0]:
y_t=np.array(y_test)
y_actual=np.argmax(y_t,axis=1)

In [62]:
y_actual

array([2, 1, 2, 1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 1, 2, 2,
       2, 1, 1, 2, 2, 0, 2, 2, 1, 2, 1, 1, 0, 0, 2, 1, 0, 2, 0, 0, 0, 1,
       2])

In [65]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_actual, y_p)
cm

array([[12,  0,  0],
       [ 0, 11,  1],
       [ 0,  3, 18]])

In [64]:
accuracy_score(y_actual, y_p)

0.9111111111111111